In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, make_scorer


In [65]:
levels = pd.read_csv("estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.csv")
classification = pd.read_csv("Obesity Classification.csv")
dataset = pd.read_csv("ObesityDataSet.csv")

## First Dataset (estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.csv) Data Brief Inspection

In [66]:
levels.head()

,Height,Weight,family_history_with_overweight,SCC,MTRANS_Walking,FAVC_z,FCVC_minmax,NCP_z,CAEC_minmax,CH2O_minmax,FAF_minmax,TUE_z,CALC_z,Age_bin_minmax,NObeyesdad
0,1.62,64.0,1,0,0,2.766876,0.5,0.404704,0.333333,0.5,0.000000,0.550985,1.439033,0.25,1
1,1.52,56.0,1,1,0,2.766876,1.0,0.404704,0.333333,1.0,1.000000,1.092724,0.516552,0.25,1
2,1.80,77.0,1,0,0,2.766876,0.5,0.404704,0.333333,0.5,0.666667,0.550985,2.472136,0.50,1
3,1.80,87.0,0,0,1,2.766876,1.0,0.404704,0.333333,0.5,0.666667,1.092724,2.472136,0.75,2
4,1.78,89.8,0,0,0,2.766876,0.5,2.164116,0.333333,0.5,0.000000,1.092724,0.516552,0.50,3


In [67]:
levels.describe(include="all")

,Height,Weight,family_history_with_overweight,SCC,MTRANS_Walking,FAVC_z,FCVC_minmax,NCP_z,CAEC_minmax,CH2O_minmax,FAF_minmax,TUE_z,CALC_z,Age_bin_minmax,NObeyesdad
count,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000,2086.000000
mean,1.702045,86.622985,0.817354,0.045062,0.026366,0.639326,0.709818,0.763670,0.379834,0.504361,0.337873,0.843202,0.855417,0.499760,3.110259
std,0.093419,26.256245,0.386469,0.207491,0.160260,0.769120,0.267493,0.645761,0.155956,0.306578,0.283687,0.537726,0.518064,0.353723,1.993832
min,1.450000,39.000000,0.000000,0.000000,0.000000,0.361418,0.000000,0.002375,0.000000,0.000000,0.000000,0.000146,0.516552,0.000000,0.000000
25%,1.630000,65.130595,1.000000,0.000000,0.000000,0.361418,0.500000,0.404704,0.333333,0.291005,0.042901,0.466622,0.516552,0.250000,1.000000
50%,1.701383,83.000000,1.000000,0.000000,0.000000,0.361418,0.695087,0.404704,0.333333,0.500000,0.333333,0.813973,0.516552,0.500000,3.000000
75%,1.768770,108.009452,1.000000,0.000000,0.000000,0.361418,1.000000,1.031717,0.333333,0.740243,0.557356,1.092724,1.439033,0.750000,5.000000
max,1.980000,165.057269,1.000000,1.000000,1.000000,2.766876,1.000000,2.164116,1.000000,1.000000,1.000000,2.194694,4.427721,1.000000,6.000000


In [68]:
levels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2086 entries, 0 to 2085
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Height                          2086 non-null   float64
 1   Weight                          2086 non-null   float64
 2   family_history_with_overweight  2086 non-null   int64  
 3   SCC                             2086 non-null   int64  
 4   MTRANS_Walking                  2086 non-null   int64  
 5   FAVC_z                          2086 non-null   float64
 6   FCVC_minmax                     2086 non-null   float64
 7   NCP_z                           2086 non-null   float64
 8   CAEC_minmax                     2086 non-null   float64
 9   CH2O_minmax                     2086 non-null   float64
 10  FAF_minmax                      2086 non-null   float64
 11  TUE_z                           2086 non-null   float64
 12  CALC_z                          20

### Conclusion: the first dataset shows that the data is clean based on the non-existence null entries on each column 

This dataset was taken from UCI library. It has been cleaned using techniques: z-score normalization, one-hot encoding, outlier removal, min-max scaling, and feature selection.

The target column 'NObeyesdad' contains the following encoding of Obesity Levels:

Insufficient_Weight: 0
Normal_Weight:1
Overweight_Level_I: 2
Overweight_Level_II: 3
Obesity_Type_I: 4
Obesity_Type_II: 5
Obesity_Type_III: 6

In [69]:
# Define features (X) and target (y)
X = levels.drop('NObeyesdad', axis=1)
y = levels['NObeyesdad']

In [70]:
# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [71]:
# Create the KNN model (k=3)
knn = KNeighborsClassifier(n_neighbors=3)

In [72]:
# Train the model
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [73]:
# Predict on the test data
y_pred = knn.predict(X_test)

In [74]:
# Train the KNN model using only the first two features (for visualization purposes)
X_train_2d = X_train.iloc[:, :2]  # Select only the first two features
X_test_2d = X_test.iloc[:, :2]

# Create the KNN model (k=3) for the 2D data
knn_2d = KNeighborsClassifier(n_neighbors=3)

# Train the model
knn_2d.fit(X_train_2d, y_train)

# Predict on the test data
y_pred_2d = knn_2d.predict(X_test_2d)

In [75]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

In [76]:
# Evaluate the 2D model
accuracy_2d = accuracy_score(y_test, y_pred_2d)
conf_matrix_2d = confusion_matrix(y_test, y_pred_2d)
class_report_2d = classification_report(y_test, y_pred_2d)


In [77]:
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.84
Confusion Matrix:
[[52  2  0  0  0  0  0]
 [12 34 15  4  1  0  0]
 [ 0  2 46  7  1  0  0]
 [ 0  3  2 39  4  0  0]
 [ 0  1  2  6 57  5  0]
 [ 0  0  0  0  0 67  0]
 [ 0  0  0  0  0  0 56]]
Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.96      0.88        54
           1       0.81      0.52      0.63        66
           2       0.71      0.82      0.76        56
           3       0.70      0.81      0.75        48
           4       0.90      0.80      0.85        71
           5       0.93      1.00      0.96        67
           6       1.00      1.00      1.00        56

    accuracy                           0.84       418
   macro avg       0.84      0.84      0.83       418
weighted avg       0.84      0.84      0.83       418



In [78]:
print(f"Accuracy with 2 features: {accuracy_2d:.2f}")
print("Confusion Matrix (2D):")
print(conf_matrix_2d)
print("Classification Report (2D):")
print(class_report_2d)

Accuracy with 2 features: 0.82
Confusion Matrix (2D):
[[53  1  0  0  0  0  0]
 [ 3 57  6  0  0  0  0]
 [ 1  5 39 11  0  0  0]
 [ 0  0  5 38  5  0  0]
 [ 0  0  4 15 46  5  1]
 [ 0  0  0  0  6 57  4]
 [ 0  0  0  0  0  3 53]]
Classification Report (2D):
              precision    recall  f1-score   support

           0       0.93      0.98      0.95        54
           1       0.90      0.86      0.88        66
           2       0.72      0.70      0.71        56
           3       0.59      0.79      0.68        48
           4       0.81      0.65      0.72        71
           5       0.88      0.85      0.86        67
           6       0.91      0.95      0.93        56

    accuracy                           0.82       418
   macro avg       0.82      0.83      0.82       418
weighted avg       0.83      0.82      0.82       418



In [79]:
#create the pipline for each model (KNN, logistic regression, decision tree classifier)

pipeline_decision_tree = Pipeline([("scaler", MinMaxScaler()), ("dt", DecisionTreeClassifier())])

decision_tree_params = {"dt__max_features":[0.2, 0.5, 0.7], "dt__min_samples_leaf":[1, 2, 3, 5, 10], "dt__max_depth":[5, 10, 15]}
f1 = make_scorer(f1_score , average='macro') #function to test on f1_score

#GridSearchCV allow you to optimize the hyperparameters of your model
grid_decision_tree = GridSearchCV(pipeline_decision_tree, decision_tree_params, cv=5, verbose=True, refit=True, n_jobs=-1,scoring=f1)

grid_decision_tree.fit(X_train, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('dt', DecisionTreeClassifier())]),
             n_jobs=-1,
             param_grid={'dt__max_depth': [5, 10, 15],
                         'dt__max_features': [0.2, 0.5, 0.7],
                         'dt__min_samples_leaf': [1, 2, 3, 5, 10]},
             scoring=make_scorer(f1_score, average=macro), verbose=True)

In [80]:
grid_decision_tree.cv_results_

{'mean_fit_time': array([0.01162977, 0.0085537 , 0.00799112, 0.00772662, 0.01195779,
        0.0115592 , 0.01234999, 0.00962105, 0.00984163, 0.01249704,
        0.00563087, 0.0101521 , 0.00972209, 0.01346574, 0.01739783,
        0.00675488, 0.0076951 , 0.00691009, 0.00746837, 0.00616021,
        0.01051188, 0.01010585, 0.01369224, 0.00891128, 0.00802164,
        0.01128116, 0.01058831, 0.01115565, 0.01108732, 0.01183457,
        0.00789795, 0.00733552, 0.00652428, 0.01057796, 0.00735555,
        0.0121017 , 0.00894365, 0.0080658 , 0.01446357, 0.00866723,
        0.01305971, 0.01730943, 0.01296329, 0.01488829, 0.01070113]),
 'std_fit_time': array([0.0038603 , 0.0017897 , 0.0010624 , 0.00265367, 0.00160031,
        0.00273508, 0.00409834, 0.00068034, 0.00427802, 0.01154142,
        0.00170747, 0.00193841, 0.00131942, 0.0018488 , 0.01388774,
        0.00364938, 0.00533426, 0.00197759, 0.00229871, 0.00393064,
        0.00054056, 0.00056517, 0.00868691, 0.00124961, 0.00157273,
        0.001

In [81]:
dt_best_estimator = grid_decision_tree.best_estimator_
print(dt_best_estimator)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('dt', DecisionTreeClassifier(max_depth=15, max_features=0.7))])


In [82]:
dt_best_score= grid_decision_tree.best_score_
print(dt_best_score)

0.905176443227219


In [83]:
y_pred = dt_best_estimator.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.91      0.88        54
           1       0.82      0.77      0.80        66
           2       0.78      0.80      0.79        56
           3       0.76      0.79      0.78        48
           4       0.87      0.83      0.85        71
           5       0.95      0.94      0.95        67
           6       1.00      1.00      1.00        56

    accuracy                           0.86       418
   macro avg       0.86      0.86      0.86       418
weighted avg       0.86      0.86      0.86       418



In [84]:
from sklearn.neighbors import KNeighborsClassifier

pipeline_knn = Pipeline([("scaler", MinMaxScaler()), ("knn", KNeighborsClassifier())])

knn_params = {
    "knn__n_neighbors": [3, 5, 7, 9, 11],
    "knn__metric": ['euclidean', 'manhattan'],
    "knn__weights": ['uniform', 'distance']
}

f1 = make_scorer(f1_score, average='macro')

grid_knn = GridSearchCV(pipeline_knn, knn_params, cv=5, verbose=True, refit=True, n_jobs=-1, scoring=f1)

grid_knn.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__metric': ['euclidean', 'manhattan'],
                         'knn__n_neighbors': [3, 5, 7, 9, 11],
                         'knn__weights': ['uniform', 'distance']},
             scoring=make_scorer(f1_score, average=macro), verbose=True)

In [85]:
best_estimator_knn = grid_knn.best_estimator_   
print(best_estimator_knn)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('knn',
                 KNeighborsClassifier(metric='manhattan', weights='distance'))])


In [86]:
best_scores_knn = grid_knn.best_score_
print(best_scores_knn)

0.8315789536312991


In [87]:
y_pred_knn = best_estimator_knn.predict(X_test)
print(classification_report(y_test, y_pred_knn))

              precision    recall  f1-score   support

           0       0.83      0.91      0.87        54
           1       0.77      0.61      0.68        66
           2       0.77      0.86      0.81        56
           3       0.71      0.75      0.73        48
           4       0.91      0.87      0.89        71
           5       0.99      1.00      0.99        67
           6       0.97      1.00      0.98        56

    accuracy                           0.86       418
   macro avg       0.85      0.86      0.85       418
weighted avg       0.86      0.86      0.85       418



In [88]:
#logistic regression

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [ ]:
pipeline_lr = Pipeline([
    ("scaler", MinMaxScaler()),
    ("lr", LogisticRegression())])

lr_params = {
    'lr__penalty': ['l1', 'l2', 'elasticnet'],
    'lr__C': [0.01, 0.1, 1, 10, 100],
    'lr__max_iter':[100,1000,100000,10000],
    'lr__solver': ['lbfgs', 'saga'] 
}
grid_lr = GridSearchCV(pipeline_lr, lr_params, cv=5, verbose=True, refit=True, n_jobs=-1, scoring=f1)
grid_lr.fit(X_train, y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits


In [49]:
best_estimator_lr = grid_lr.best_estimator_   
print(best_estimator_lr)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('lr',
                 LogisticRegression(C=100, max_iter=100000, penalty='l1',
                                    solver='saga'))])


In [50]:
best_scores_lr = grid_lr.best_score_
print(best_scores_lr)

0.9630036627245842


In [51]:
y_pred_lr = best_estimator_lr.predict(X_test)
print(classification_report(y_test, y_pred_lr))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       0.98      0.98      0.98        66
           2       0.95      0.93      0.94        56
           3       0.94      0.92      0.93        48
           4       0.97      0.99      0.98        71
           5       0.99      1.00      0.99        67
           6       1.00      1.00      1.00        56

    accuracy                           0.98       418
   macro avg       0.97      0.97      0.97       418
weighted avg       0.98      0.98      0.98       418



In [52]:
from sklearn.multiclass import OneVsRestClassifier , OneVsOneClassifier

In [53]:
#one vs rest
dt_pipeline_tree_ovr = Pipeline([('scaler', MinMaxScaler()),
                ('dt_ovr',
                 OneVsRestClassifier(DecisionTreeClassifier(max_depth=15, max_features=0.7
                                                            ,
                                        min_samples_leaf=3)))])

dt_pipeline_tree_ovr.fit(X_train, y_train)

y_pred_ovr = dt_pipeline_tree_ovr.predict(X_test)

print(classification_report(y_test, y_pred_ovr))

              precision    recall  f1-score   support

           0       0.98      0.93      0.95        54
           1       0.90      0.82      0.86        66
           2       0.89      0.84      0.86        56
           3       0.86      0.92      0.89        48
           4       0.95      0.85      0.90        71
           5       0.99      0.99      0.99        67
           6       0.77      1.00      0.87        56

    accuracy                           0.90       418
   macro avg       0.90      0.90      0.90       418
weighted avg       0.91      0.90      0.90       418



In [54]:
#one vs one
dt_pipeline_tree_ovo = Pipeline([('scaler', MinMaxScaler()),
                ('dt_ovo',
                 OneVsOneClassifier(DecisionTreeClassifier(max_depth=15, max_features=0.7
                                                            ,
                                        min_samples_leaf=3)))])

dt_pipeline_tree_ovo.fit(X_train, y_train)

y_pred_ovo = dt_pipeline_tree_ovo.predict(X_test)

print(classification_report(y_test, y_pred_ovo))

              precision    recall  f1-score   support

           0       0.90      0.87      0.89        54
           1       0.88      0.65      0.75        66
           2       0.77      0.88      0.82        56
           3       0.74      0.90      0.81        48
           4       0.94      0.92      0.93        71
           5       0.97      1.00      0.99        67
           6       0.98      1.00      0.99        56

    accuracy                           0.89       418
   macro avg       0.88      0.89      0.88       418
weighted avg       0.89      0.89      0.88       418



In [55]:
pipeline_knn_ovr = Pipeline([('scaler', MinMaxScaler()),
                ('knn_ovr',
                 OneVsRestClassifier(KNeighborsClassifier(metric='manhattan', n_neighbors=3,
                                      weights='distance')))])

pipeline_knn_ovr.fit(X_train, y_train)
knn_y_pred_ovr = pipeline_knn_ovr.predict(X_test)
print(classification_report(y_test, knn_y_pred_ovr))

              precision    recall  f1-score   support

           0       0.84      0.91      0.88        54
           1       0.76      0.64      0.69        66
           2       0.78      0.80      0.79        56
           3       0.70      0.77      0.73        48
           4       0.88      0.85      0.86        71
           5       0.99      1.00      0.99        67
           6       0.97      1.00      0.98        56

    accuracy                           0.85       418
   macro avg       0.85      0.85      0.85       418
weighted avg       0.85      0.85      0.85       418



In [56]:
pipeline_knn_ovo = Pipeline([('scaler', MinMaxScaler()),
                  ('knn_ovo',
                 OneVsOneClassifier(KNeighborsClassifier(metric='manhattan', n_neighbors=3,
                                      weights='distance')))])

pipeline_knn_ovo.fit(X_train, y_train)
knn_y_pred_ovo = pipeline_knn_ovo.predict(X_test)
print(classification_report(y_test, knn_y_pred_ovo))

              precision    recall  f1-score   support

           0       0.84      0.91      0.88        54
           1       0.78      0.64      0.70        66
           2       0.77      0.84      0.80        56
           3       0.73      0.77      0.75        48
           4       0.88      0.85      0.86        71
           5       0.99      1.00      0.99        67
           6       0.97      1.00      0.98        56

    accuracy                           0.86       418
   macro avg       0.85      0.86      0.85       418
weighted avg       0.86      0.86      0.85       418



In [57]:
pipeline_lr_ovr = Pipeline([('scaler', MinMaxScaler()),
                ('lr_ovr',
                 OneVsRestClassifier(LogisticRegression(C=100, max_iter=100000, penalty='l1',
                                    solver='saga')))])

pipeline_lr_ovr.fit(X_train, y_train)
y_pred_lr_ovr = pipeline_lr_ovr.predict(X_test)
print(classification_report(y_test, y_pred_lr_ovr))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        54
           1       0.82      0.47      0.60        66
           2       0.58      0.66      0.62        56
           3       0.47      0.50      0.48        48
           4       0.62      0.72      0.67        71
           5       0.84      0.88      0.86        67
           6       0.97      1.00      0.98        56

    accuracy                           0.75       418
   macro avg       0.75      0.75      0.74       418
weighted avg       0.76      0.75      0.74       418



In [58]:
pipeline_lr_ovo = Pipeline([('scaler', MinMaxScaler()),
                ('lr_ovo',
                 OneVsOneClassifier(LogisticRegression(C=100, max_iter=100000, penalty='l1',
                                    solver='saga')))])

pipeline_lr_ovo.fit(X_train, y_train)
y_pred_lr_ovo = pipeline_lr_ovo.predict(X_test)
print(classification_report(y_test, y_pred_lr_ovo))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        54
           1       0.98      0.98      0.98        66
           2       0.95      0.95      0.95        56
           3       0.96      0.94      0.95        48
           4       0.99      0.99      0.99        71
           5       0.99      1.00      0.99        67
           6       1.00      1.00      1.00        56

    accuracy                           0.98       418
   macro avg       0.98      0.98      0.98       418
weighted avg       0.98      0.98      0.98       418



In [59]:
pipelines = {
    'OvR_KNN': Pipeline([
        ("scaler", MinMaxScaler()),
        ("knn", OneVsRestClassifier(KNeighborsClassifier()))
    ]),
    'OvO_KNN': Pipeline([
        ("scaler", MinMaxScaler()),
        ("knn", OneVsOneClassifier(KNeighborsClassifier()))
    ]),
    'OvR_DT': Pipeline([
        ("scaler", MinMaxScaler()),
        ("dt", OneVsRestClassifier(DecisionTreeClassifier(random_state=42)))
    ]),
    'OvO_DT': Pipeline([
        ("scaler", MinMaxScaler()),
        ("dt", OneVsOneClassifier(DecisionTreeClassifier(random_state=42)))
    ]),
    'OvR_LR': Pipeline([
        ("scaler", MinMaxScaler()),
        ("lr", OneVsRestClassifier(LogisticRegression(penalty=None)))
    ]),
    'OvO_LR': Pipeline([
        ("scaler", MinMaxScaler()),
        ("lr", OneVsOneClassifier(LogisticRegression(penalty=None)))
    ])
}

In [60]:
param_grids = {
    'OvR_KNN': {
        'knn__estimator__n_neighbors': [3, 5, 7, 9, 11],
        'knn__estimator__metric': ['euclidean', 'manhattan'],
        'knn__estimator__weights': ['uniform', 'distance']
    },
    'OvO_KNN': {
        'knn__estimator__n_neighbors': [3, 5, 7, 9, 11],
        'knn__estimator__metric': ['euclidean', 'manhattan'],
        'knn__estimator__weights': ['uniform', 'distance']
    },
    'OvR_DT': {
        'dt__estimator__max_features': [0.2, 0.5, 0.7],
        'dt__estimator__min_samples_leaf': [1, 2, 3, 5, 10],
        'dt__estimator__max_depth': [5, 10, 15]
    },
    'OvO_DT': {
        'dt__estimator__max_features': [0.2, 0.5, 0.7],
        'dt__estimator__min_samples_leaf': [1, 2, 3, 5, 10],
        'dt__estimator__max_depth': [5, 10, 15]
    },
    'OvR_LR': {
        'lr__estimator__max_iter': [100, 1000, 100000, 10000],
        'lr__estimator__solver': ['lbfgs', 'saga']
    },
    'OvO_LR': {
        'lr__estimator__max_iter': [100, 1000, 100000, 10000],
        'lr__estimator__solver': ['lbfgs', 'saga']
    }
}

In [63]:
best_estimators = {}
best_scores = {}

for name, pipeline in pipelines.items():
    grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, verbose=True, refit=True, n_jobs=-1, scoring=f1)
    grid_search.fit(X_train, y_train)
    best_estimators[name] = grid_search.best_estimator_
    best_scores[name] = grid_search.best_score_

for name, estimator in best_estimators.items():
    print(f"{name}: {estimator}")

for name, score in best_scores.items():
    print(f"{name}: {score}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 45 candidates, totalling 225 fits
Fitting 5 folds for each of 45 candidates, totalling 225 fits
Fitting 5 folds for each of 8 candidates, totalling 40 fits


c:\Users\hp\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
40 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\hp\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\hp\anaconda3\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\hp\anaconda3\lib\site-packages\sklearn\multiclass.py", line 337, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs)(
  File "c:\Users\hp\anacon

NotFittedError: All estimators failed to fit

In [62]:
!pip install eli5

  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=a1cea41409036dc914420b03b187f556f66a96586b07d889bc6e8702e981b7be
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\7b\26\a5\8460416695a992a2966b41caa5338e5e7fcea98c9d032d055c
Successfully built eli5


## Second Dataset (Obesity Classification.csv) Data Brief Inspection

In [4]:
classification.head()

,ID,Age,Gender,Height,Weight,BMI,Label
0,1,25,Male,175,80,25.3,Normal Weight
1,2,30,Female,160,60,22.5,Normal Weight
2,3,35,Male,180,90,27.3,Overweight
3,4,40,Female,150,50,20.0,Underweight
4,5,45,Male,190,100,31.2,Obese


In [16]:
classification.describe(include="all")

,ID,Age,Gender,Height,Weight,BMI,Label
count,108.000000,108.000000,108,108.000000,108.000000,108.000000,108
unique,NaN,NaN,2,NaN,NaN,NaN,4
top,NaN,NaN,Male,NaN,NaN,NaN,Underweight
freq,NaN,NaN,56,NaN,NaN,NaN,47
mean,56.046296,46.555556,NaN,166.574074,59.490741,20.549074,NaN
std,31.917939,24.720620,NaN,27.873615,28.856233,7.583818,NaN
min,1.000000,11.000000,NaN,120.000000,10.000000,3.900000,NaN
25%,28.750000,27.000000,NaN,140.000000,35.000000,16.700000,NaN
50%,56.500000,42.500000,NaN,175.000000,55.000000,21.200000,NaN
75%,83.250000,59.250000,NaN,190.000000,85.000000,26.100000,NaN


In [9]:
classification.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      108 non-null    int64  
 1   Age     108 non-null    int64  
 2   Gender  108 non-null    object 
 3   Height  108 non-null    int64  
 4   Weight  108 non-null    int64  
 5   BMI     108 non-null    float64
 6   Label   108 non-null    object 
dtypes: float64(1), int64(4), object(2)
memory usage: 6.0+ KB


### Conclusion: the second dataset shows that the data is clean based on the non-existence null entries on each column 

## Third Dataset (ObesityDataSet.csv) Data Brief Inspection

In [5]:
dataset.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [15]:
dataset.describe(include="all")

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
count,2111,2111.000000,2111.000000,2111.000000,2111,2111,2111.000000,2111.000000,2111,2111,2111.000000,2111,2111.000000,2111.000000,2111,2111,2111
unique,2,NaN,NaN,NaN,2,2,NaN,NaN,4,2,NaN,2,NaN,NaN,4,5,7
top,Male,NaN,NaN,NaN,yes,yes,NaN,NaN,Sometimes,no,NaN,no,NaN,NaN,Sometimes,Public_Transportation,Obesity_Type_I
freq,1068,NaN,NaN,NaN,1726,1866,NaN,NaN,1765,2067,NaN,2015,NaN,NaN,1401,1580,351
mean,NaN,24.312600,1.701677,86.586058,NaN,NaN,2.419043,2.685628,NaN,NaN,2.008011,NaN,1.010298,0.657866,NaN,NaN,NaN
std,NaN,6.345968,0.093305,26.191172,NaN,NaN,0.533927,0.778039,NaN,NaN,0.612953,NaN,0.850592,0.608927,NaN,NaN,NaN
min,NaN,14.000000,1.450000,39.000000,NaN,NaN,1.000000,1.000000,NaN,NaN,1.000000,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,19.947192,1.630000,65.473343,NaN,NaN,2.000000,2.658738,NaN,NaN,1.584812,NaN,0.124505,0.000000,NaN,NaN,NaN
50%,NaN,22.777890,1.700499,83.000000,NaN,NaN,2.385502,3.000000,NaN,NaN,2.000000,NaN,1.000000,0.625350,NaN,NaN,NaN
75%,NaN,26.000000,1.768464,107.430682,NaN,NaN,3.000000,3.000000,NaN,NaN,2.477420,NaN,1.666678,1.000000,NaN,NaN,NaN


In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

### Conclusion: the third dataset shows that the data is clean based on the non-existence null entries on each column 

The attributes related with eating habits are: Frequent consumption of high caloric food (FAVC), Frequency of consumption of vegetables (FCVC), Number of main meals (NCP), Consumption of food between meals (CAEC), Consumption of water daily (CH20), and Consumption of alcohol (CALC). The attributes related with the physical condition are: Calories consumption monitoring (SCC), Physical activity frequency (FAF), Time using technology devices (TUE), Transportation used (MTRANS)
variables obtained :
Gender, Age, Height and Weight.

NObesity values are:

•Underweight Less than 18.5
•Normal 18.5 to 24.9
•Overweight 25.0 to 29.9
•Obesity I 30.0 to 34.9
•Obesity II 35.0 to 39.9
•Obesity III Higher than 40

